### 게임 리뷰 데이터 추출하기

In [1]:
! pip install koreanize-matplotlib
! pip install matplotlib
! pip install numpy
! pip install pandas
! pip install openpyxl
! pip install selenium
! pip install webdriver-manager
! pip install wordcloud
! pip install konlpy
! pip install apyori

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.9/7.9 MB 24.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.7/9.7 MB 68.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 486.3/486.3 kB 32.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 19.4/19.4 MB 55.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 493.8/493.8 kB 23.9 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for apyori: filename=apyori-1.1.2-py3-none-any.whl size=5954 sha256=3d50a780f0f69e839a95d1e91aa1ea8044b90ac926afebe779aca677c1534bc9
  Stored in directory: /root/.cache/pip/wheels/c4/1a/79/20f55c470a50bb3702a8cb7c94d8ada15573538c7f4baebe2d
Successfully built apyori


#### minimap 에서 리뷰를 추출해오는 함수 생성

In [ ]:
def get_review(game):
    from selenium import webdriver
    from selenium.webdriver.chrome.service import Service
    from selenium.webdriver.common.by import By
    from selenium.webdriver.support.ui import WebDriverWait
    from selenium.webdriver.support import expected_conditions as EC
    from webdriver_manager.chrome import ChromeDriverManager
    from selenium.common.exceptions import TimeoutException
    from selenium.webdriver.common.keys import Keys
    import pyperclip
    import time

    # Chrome 드라이버 설정
    service = Service(ChromeDriverManager().install())
    options = webdriver.ChromeOptions()
    driver = webdriver.Chrome(service=service, options=options)

    # 게임 리뷰 데이터를 저장할 reviews 리스트 생성
    reviews = []

    try:
        # 웹페이지 URL
        url = f"https://minimap.net/game/{game}/review?l=kr"

        # 웹페이지 로드
        driver.get(url)

        # 로그인 버튼 찾기 및 클릭
        login_button = WebDriverWait(driver, 10).until(
            EC.element_to_be_clickable((By.CSS_SELECTOR, ".C-button .c-button-item.c-color-primary"))  # 실제 로그인 버튼의 CSS 선택자로 변경 필요
        )
        login_button.click()

        login_button2 = WebDriverWait(driver, 10).until(
            EC.element_to_be_clickable((By.CSS_SELECTOR, ".pop-sign-up .button-w .login-button-w .login-button"))  # 실제 로그인 버튼의 CSS 선택자로 변경 필요
        )
        login_button2.click()

        # 로그인 팝업 창이 나타날 때까지 대기
        WebDriverWait(driver, 10).until(
            EC.presence_of_element_located((By.CSS_SELECTOR, "#popupWrapper .popup-wrap-background .popup-dialog-wrap"))  # 실제 팝업 창의 CSS 선택자로 변경 필요
        )

        # 로그인 정보 입력
        id_field = driver.find_element(By.ID, "sign-in-email-input")  # 실제 사용자 이름 필드의 ID로 변경 필요
        pw_field = driver.find_element(By.ID, "sign-in-password-input")  # 실제 비밀번호 필드의 ID로 변경 필요

        pyperclip.copy("lhe9614@gmail.com")
        id_field.send_keys(Keys.CONTROL, 'v')

        pyperclip.copy("!Moon@126419#")
        pw_field.send_keys(Keys.CONTROL, 'v')

        time.sleep(2)

        # 로그인 제출 버튼 클릭
        submit_button = driver.find_element(By.CSS_SELECTOR, ".pop-sign-in .button-w .C-round-button")  # 실제 제출 버튼의 CSS 선택자로 변경 필요
        submit_button.click()

        # 로그인 완료 대기 (예: 특정 요소가 사라질 때까지)
        WebDriverWait(driver, 10).until(
            EC.invisibility_of_element_located((By.CSS_SELECTOR, "#popupWrapper .popup-wrap-background .popup-dialog-wrap"))
        )

        # 로그인 완료 후 페이지가 완전히 로드될 때까지 대기
        WebDriverWait(driver, 20).until(
            EC.presence_of_element_located((By.CLASS_NAME, "review-list"))
        )

        # 리뷰 페이지로 다시 이동
        driver.get(url)

        # 리뷰 컨테이너 요소 찾기
        review_container = WebDriverWait(driver, 20).until(
            EC.presence_of_element_located((By.CLASS_NAME, "review-list"))
        )

        # 컨테이너 끝까지 스크롤
        driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")

        # 스크롤 반복해서 내리기
        ## 최대 반복 수 설정
        max_cnt = 100

        ## 이전 높이 계산
        old_height = driver.execute_script("return document.body.scrollHeight")

        cnt = 0
        while True:
            ## 스크롤 다운
            driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")

            ## 대기
            time.sleep(0.5)

            ## 새로운 높이 계산
            new_height = driver.execute_script("return document.body.scrollHeight")
            ## 더 이상 스크롤되지 않으면 종료
            if new_height == old_height:
                break
            ## 새로운 높이 계산 값을 이전 높이 변수에 저장장
            old_height = new_height

            ## 카운트 계산
            if cnt == max_cnt:
                break
            cnt += 1

        # 모든 "post-content" 클래스를 가진 div 요소 찾기
        post_contents = driver.find_elements(By.CLASS_NAME, "post-content")

        # 각 "post-content" div의 텍스트 내용 추출
        for post in post_contents:
            review_text = post.text.strip()  # div 내의 모든 텍스트를 가져옴
            reviews.append(review_text)

    except Exception as e:
        print(f"오류 발생, {e}")

    finally:
        # 브라우저 종료
        driver.quit()

    return reviews

#### game_list 에 해당하는 게임들의 리뷰 추출

In [ ]:
import pandas as pd

# 게임의 리뷰를 가져오기 위한 game_list 라는 변수들의 list 생성
game_list = {
    ('쿠키런킹덤', 'com-devsisters-ck'),
    ('던전앤파이터', 'dungeon-and-fighters'),
    ('메이플스토리', 'maplestory'),
    ('월드오브워크래프트', 'world-of-warcraft'),
    ('블레이드앤소울', 'blade-soul'),
    ('마비노기', 'mabinogi'),
    ('리그오브레전드', 'league-of-legends'),
    ('펜타스톰', 'arena-of-valor'),
    ('펜타스톰', 'com-netmarble-penta-gm'),
    ('도타2', 'dota-2'),
    ('브롤스타즈', 'com-supercell-brawlstars'),
    ('서든어택', 'sudden-attack'),
    ('배틀그라운드', 'playerunknowns-battlegrounds'),
    ('배틀그라운드', 'com-pubg-krmobile'),
    ('오버워치', 'overwatch'),
    ('발로란트', 'valorant'),
    ('클래시오브클랜', 'com-supercell-clashofclans'),
    ('클래시로얄', 'com-supercell-clashroyale-gm'),
    ('스타크래프트', 'starcraft'),
    ('피파온라인', 'fifa-online-4'),
    ('피파23', 'ea-sports-fifa-23'),
    ('캔디크러쉬사가', 'com-king-candycrushsaga'),
    ('마인크래프트', 'minecraft'),
    ('로블록스', 'roblox'),
    ('로블록스', 'com-roblox-client-gm'),
    ('카트라이더', 'kartrider'),
    ('테일즈런너', 'tales-runner'),
    ('크레이지아케이드', 'crazy-arcade'),
    ('모여봐요동물의숲', 'animal-crossing-new-horizons'),
    ('모두의마블', 'com-cjenm-modoomarblekakao-gm'),
    ('포켓몬스터', 'pokemon-scarlet')
}

# 모든 리뷰를 저장할 리스트 생성
all_reviews = []

for game_name, game_id in game_list:
    game_reviews = get_review(game_id)

    for review in game_reviews:
        all_reviews.append({
            "Game": game_name,
            "Review": review
        })

# 리뷰 데이터를 데이터프레임으로 변환
game_reviews_df = pd.DataFrame(all_reviews)

# 데이터프레임 출력
print(game_reviews_df)

# CSV 파일로 저장
game_reviews_df.to_csv('game_reviews.csv', index=False)



      Game                                             Review
0     로블록스                     이 게임 하나의 모든 장르에 게임이 있어서 정말 추천함
1     로블록스                                         진짜 할 거 없어요
2     로블록스  게임이라기 보단 게임들이 모여있는 플렛폼에 더 가깝다.\n하지만 게임 저작권 문제해...
3     로블록스  진짜 모든 사람에게 추천드려요!\n왜냐하면 모든 장르의 게임이 다있거든요!\n 컴퓨...
4     로블록스       와!로블록스아시는구나!겁나갓겜입니다\n같은 이미지가 있어서 그렇지 생각보단 ㄱㅊ
..     ...                                                ...
687  테일즈런너  얼라시절 정말 열심히 했었다\n그때쯤 유키가 신캐였던가 그랬지\n난 컨트롤이 그때도...
688  테일즈런너  누구는 추억 보정 잔뜩 들어간 게임이라고 말하지만\n그 당시 이 게임이 가진 매력과...
689  테일즈런너                                             재밌는 테런
690  테일즈런너                                                재밋다
691  테일즈런너               똥망겜이라고 하지만 누구보다 열심히 플레이 했던\n 어린 나...

[692 rows x 2 columns]


### 시각화

In [2]:
import pandas as pd
import os

# 현재 디렉터리에 'data' 디렉터리가 있는지 확인
if os.path.exists("data") and os.path.isdir("data"):
    data_path = "data/"
else:
    # 상위 디렉터리 '../data'를 확인
    data_path = "../data/"

# 데이터 불러오기
data_review = pd.read_csv(data_path + 'game_reviews.csv')
print(data_review)
meta = pd.read_csv(data_path + 'game_meta.csv')
meta.dropna(inplace=True)
print(meta)
# 데이터 통합
meta.rename(columns={"name": "Game"}, inplace=True)
data_review = pd.merge(data_review, meta, on="Game", how="outer")
data_review.dropna(inplace=True)
data_review


      Game                                             Review
0     피파23                                     라스트 FIFA, 그 이름
1     피파23  EA FC 24 티저 나오자마자 별로 기대 안되길래 언젠간 해봐야지 했던 축구 게임...
2     피파23  엑박판으로 플레이 중이지만 PC판은 실행이 안됨.\n EA 패망의 서막을 알리는 게...
3     피파23  축구는 종교다.\nFIFA 23 | Official Launch Trailer | ...
4     피파23  최고의 손님 접대용 게임.\n하이퍼모션2가 적용되면서 움직임의 사실성이 훨씬 좋아졌...
...    ...                                                ...
1737  발로란트  재미는 있다... 그런데 중간중간 쉬는 텀없이 빠르게 진행되서 그런지 5판~10판 ...
1738  발로란트                                           오우 싸발적이고
1739  발로란트  역시 게임은 처음 나왔을 때가 젤 재밌다..\n클로즈베타때 그렇게 했는데\n정식출시...
1740  발로란트  라이엇이 만든 FPS\n롤이랑 세계관이 다르지만 꽤 잘 만들긴 했다.\n우리나라에서...
1741  발로란트                  클래식fps와 하이퍼fps가 결합한게임중 가장 교과서적이다.

[1742 rows x 2 columns]
         name main_age  difficulty  playtime  achievement genre
0      던전앤파이터    25_29        3.10     79.19         2.27   RPG
1         도타2    25_29        4.00     91.70         3.10   AOS
2        로블록스    06_12        2.50     

,Game,Review,main_age,difficulty,playtime,achievement,genre
0,던전앤파이터,넥슨 너네는 던파 도트팀 사운드팀 진짜 잘해줘야된다,25_29,3.10,79.19,2.27,RPG
1,던전앤파이터,메이플 상위버전 그래픽은 옛날이라구리지만 살아있다는게 신기함.\n여전히 뽑아서 쓰고...,25_29,3.10,79.19,2.27,RPG
2,던전앤파이터,지금의 이미지가 구려서 그렇지 게임은 정말 재미있다. 처음 나왔을 때 던전앤드래곤을...,25_29,3.10,79.19,2.27,RPG
3,던전앤파이터,게임이 어둑섬까지 잘 되다가 안개신 레이드에서 불쾌감을 나타낼만한 환경으로 조성이 ...,25_29,3.10,79.19,2.27,RPG
4,던전앤파이터,뉴비여서 고인물의입장은 잘 모르겠지만\n고인물들이 뉴비유입을 막는건 말이안된다생각\...,25_29,3.10,79.19,2.27,RPG
...,...,...,...,...,...,...,...
1706,피파23,커리어 모드 혁신이 필요함,13_18,3.06,60.00,3.23,스포츠
1707,피파23,쇼케이스때 지성이형 본것만해도 만족\n게임도 만족! 한글화도 감사. 축구게임은 앞으...,13_18,3.06,60.00,3.23,스포츠
1708,피파23,축구팬으로서 솔직히 재미없다고 하면 거짓말이지만\n커리어모드만 하는 나로서는 아쉬운...,13_18,3.06,60.00,3.23,스포츠
1709,피파23,월드컵과\n마지막 EA의 피파,13_18,3.06,60.00,3.23,스포츠


In [3]:
data_review.groupby("main_age").count()['Review']

main_age
06_12    716
13_18    281
19_24     17
25_29    675
Name: Review, dtype: int64

#### 형태소 분석기를 이용하여 리뷰의 단어 추출

In [15]:
from konlpy.tag import Kkma
from collections import Counter
import re

def generate_nouns_dict(dataframe, col_name):
    # 모든 리뷰를 담을 딕셔너리 생성
    nouns_dict = {}

    # Okt 형태소 분석기 초기화
    kkma = Kkma()

    # 불용어 지정
    stopwords = ["때문", "그때", "정말", "매우", "게임", "하다", "노기", "포켓", "정도", "내가", "10", "계속", "지금", "발로", "오버", "라이",
                "마비", "카트", "와우", "이드", "구들", "나름", "만큼", "가지", "카스", "17"]

    # 주연령별로 리뷰 분석
    for age in dataframe[col_name].unique():
        # 해당 게임의 모든 리뷰 추출
        game_reviews = dataframe[dataframe[col_name] == age]['Review']

        # 모든 리뷰에서 명사 추출
        nouns_list = []
        for review in game_reviews:
            if isinstance(review, str):
                review_edit = re.sub("[^0-9a-zA-Z가-힣\\s+]", "", review)
                review_edit_pos = kkma.pos(review_edit)
                sub_list = []
                for word, pos in review_edit_pos:
                    if len(word) == 1 or word in stopwords:
                        continue
                    if pos.startswith('N'):  # 태그가 N으로 시작하는 모든 명사 추출
                        sub_list.append(word)
                # nouns_list.extend(sub_list)
                nouns_list.append(sub_list)
        
        # 게임별 명사 리스트 저장
        nouns_dict[age] = nouns_list
        
        print(f"{age} 의 리뷰의 등장하는 명사> ")
        # print(set(nouns_list))
        print(nouns_list[0])
        print("")

    return nouns_dict


# 12세 이하와 13세 초과 리뷰를 비교하는 방안?

25_29 의 리뷰의 등장하는 명사> 
['도트', '사운드']

06_12 의 리뷰의 등장하는 명사> 
['알고리즘', '의해', '버그', '무료', '추천']

19_24 의 리뷰의 등장하는 명사> 
['평가', '필요', '가능', '플레이', '유일', '평점', '05', '지속적', '성공', '이유', '이유', '지속적', '적극', '적인', '업데이트', '랭크', '승부욕', '자극', '랭크', '살벌', '채팅', '유저', '트롤링', '다짐', '컴퓨터', '습관', '클라이언트', '역할', '사람', '스팀', '라도', '남자', '대화', '주제가', '코로나', '시국', '친구', '이야기', '기도']

13_18 의 리뷰의 등장하는 명사> 
['시대', '초등학교', '학년', '처음', '베타', '테스트', '시절', '기억', '신세계', '자유도', '완두콩', '세상', '마이크로', '소프트', '인수', '운영', '본래', '성공', '역사', '이자', '전설', '자체', '지속', '적인', '업데이트', '콘텐츠', '플레이', '이후', '제작', '가치', '시작', '평점', '요즈음', '튜브', '마인', '크래프트', '튜브', '트렌드', '노가다', '능력', '장착', '작업', '카타르시스', '존경심']



In [9]:
nouns_dict = generate_nouns_dict(data_review, "main_age")
nouns_dict

{'25_29': [['도트', '사운드'],
  ['메이', '상위', '버전', '그래픽', '옛날', '기함', '강화도', '노가다', '있음'],
  ['이미지',
   '처음',
   '드래곤',
   '연상',
   '제목',
   '스타일',
   '듯이',
   '사의',
   '거지',
   '운영',
   '유저',
   '이미지',
   '최악'],
  ['안개',
   '불쾌감',
   '환경',
   '조성',
   '안개',
   '플레이',
   '느낌',
   '아바',
   '행위',
   '플레이',
   '가능',
   '가격',
   '하면서',
   '플레이',
   '가능',
   '수준',
   '현재',
   '유저',
   '일반',
   '유저',
   '걸이',
   '기준',
   '자체',
   '기존',
   '유저',
   '일반',
   '유저',
   '분쟁',
   '포함',
   '증거물',
   '수집',
   '허위',
   '사실',
   '신고',
   '피해',
   '유저',
   '파악',
   '기존',
   '유저',
   '기준점',
   '오픈',
   '클리어',
   '기준',
   '컨텐츠',
   '노력',
   '인내',
   '필요',
   '컨트롤',
   '위주',
   '생각',
   '파이터',
   '온라인',
   '플레이',
   '기간',
   '본인',
   '유저',
   '누명',
   '유저',
   '평판',
   '행위',
   '커뮤니티',
   '내부',
   '문제',
   '발생',
   '조사'],
  ['고인물', '입장', '고인물', '유입', '생각', '더하기', '사람', '미적분', '입장', '컨텐츠', '걱정'],
  ['도트', '그래픽', '디자인', '음악'],
  ['추억',
   '세계관',
   '직업',
   '충동',
   '한데',
   '난이도',
   '장난',
   '인식',
   '2020927

#### 명사 워드클라우드 출력

In [16]:
# word cloud
from wordcloud import WordCloud
from collections import Counter
from PIL import Image 
import numpy as np 
import matplotlib.pyplot as plt
import koreanize_matplotlib 

# 한글 폰트 설정 (맥OS 기준, 윈도우의 경우 적절한 폰트 경로로 변경 필요)
# font_path = 'C:/Windows/Fonts/HMFMOLD.TTF'
font_path = "/System/Library/Fonts/Supplemental/AppleSDGothicNeo.ttc"

def generate_wordcloud(nouns_dict, save=False, prefix=""):
# 각 게임별로 워드클라우드 생성
    for k, v in nouns_dict.items():
        # 단어 빈도수 계산
        words = sum(v, [])
        counter = Counter(words)
        word_counts = dict(counter.most_common(25))
        # 마스크 이미지 가져오기
        mask_img = np.array(Image.open(data_path + "console.png"))
        
        # 워드클라우드 생성
        wordcloud = WordCloud(
            font_path=font_path,
            width=512, 
            height=512,
            mask=mask_img,
            background_color='white',
            colormap="twilight"
        ).generate_from_frequencies(word_counts)
        
        # 워드클라우드 그리기
        plt.figure(figsize=(16, 16))
        plt.imshow(wordcloud, interpolation='bilinear')
        plt.axis('off')
        plt.title(f"{k} 리뷰 워드클라우드 분석")
        if save == True:
            # 이미지 저장
            plt.savefig(f"{prefix}_{k}_wordcloud.png")
            plt.close()
        else:
            plt.show()
        
generate_wordcloud(nouns_dict, save=True, prefix="all")

#### 그래픽에 관련된 리뷰 워드클라우드 출력

In [17]:
# 데이터 로드
data_graphic_review = data_review.copy()

# 키워드 리스트
keywords = ["그래픽", "화질", "프레임", "멀미", "화면", "비디오", "모션", "퀄리티", "3D", "2D", "3인칭", "1인칭"]

# 키워드가 포함된 리뷰만 필터링
data_graphic_review['contains_keyword'] = data_graphic_review['Review'].apply(lambda x: any(keyword in str(x) for keyword in keywords))
data_graphic_review = data_graphic_review[data_graphic_review['contains_keyword']]

data_graphic_review
data_graphic_review.groupby("main_age").count()['Review']

graphic_nouns_dict = generate_nouns_dict(data_graphic_review, "main_age")
generate_wordcloud(graphic_nouns_dict, save=True, prefix="graphic")


25_29 의 리뷰의 등장하는 명사> 
['메이', '상위', '버전', '그래픽', '옛날', '기함', '강화도', '노가다', '있음']

06_12 의 리뷰의 등장하는 명사> 
['록스', '개인적', '추천', '중생', '놀림', '분명', '단점', '자주', '있음', '장점', '퀄리티', '시간', '가격', '무료', '부분', '유료', '상점', '일상', '최적화', '마인', '크래프트', '최적화', '조작', '형식', '조작']

13_18 의 리뷰의 등장하는 명사> 
['업데이트', '약간', '그래픽', '부족']



#### 네트워크 분석

In [31]:
# 네트워크 분석 시각화
import networkx as nx
import numpy as np
import pandas as pd
from matplotlib import pyplot as plt
from apyori import apriori


def generate_network_graph(key_name, save=False):
    # 스타일 적용
    plt.style.use("seaborn-v0_8-darkgrid")
    plt.rc("font", family="NanumGothic")  # 한글 폰트 설정
    
    # Apriori 결과 생성
    apri = list(apriori(graphic_nouns_dict[key_name], min_support=0.08))
    df_apri = pd.DataFrame(apri)
    
    df_apri["length"] = [len(i) for i in df_apri["items"]]
    df_apri = df_apri.loc[df_apri["length"] == 2]  # 길이가 2인 연관 항목만 선택
    
    # 그래프 생성
    G = nx.Graph()
    G.add_edges_from(df_apri["items"])
    
    # PageRank 계산
    pr = nx.pagerank(G)
    nsize = np.array(list(pr.values()))
    nsize = 2000 * (nsize - min(nsize)) / (max(nsize) - min(nsize))  # 노드 크기 정규화
    
    # 노드 색상 매핑
    ncolor = np.array(list(pr.values()))
    ncolor = (ncolor - min(ncolor)) / (max(ncolor) - min(ncolor))  # 색상 정규화
    
    pos = nx.kamada_kawai_layout(G)  # 노드 위치 설정
    
    # 그래프 시각화
    nx.draw(
        G,
        pos,
        node_size=nsize,
        node_color=ncolor,
        cmap="coolwarm",
        with_labels=True,
        font_size=10,
        font_family="NanumGothic",
        edge_color="gray",
        linewidths=1.0,
        alpha=0.9
    )
    
    # 제목 추가
    plt.title(f"{key_name}_그래픽 관련 리뷰 네트워크 시각화", fontsize=16, pad=20)
    
    # 그래프 저장 또는 표시
    if save:
        plt.savefig(f"{key_name}_network_graph.png", bbox_inches="tight")
        plt.close()
    else:
        plt.show()



In [30]:
print(plt.style.available)

['Solarize_Light2', '_classic_test_patch', '_mpl-gallery', '_mpl-gallery-nogrid', 'bmh', 'classic', 'dark_background', 'fast', 'fivethirtyeight', 'ggplot', 'grayscale', 'petroff10', 'seaborn-v0_8', 'seaborn-v0_8-bright', 'seaborn-v0_8-colorblind', 'seaborn-v0_8-dark', 'seaborn-v0_8-dark-palette', 'seaborn-v0_8-darkgrid', 'seaborn-v0_8-deep', 'seaborn-v0_8-muted', 'seaborn-v0_8-notebook', 'seaborn-v0_8-paper', 'seaborn-v0_8-pastel', 'seaborn-v0_8-poster', 'seaborn-v0_8-talk', 'seaborn-v0_8-ticks', 'seaborn-v0_8-white', 'seaborn-v0_8-whitegrid', 'tableau-colorblind10']


In [33]:
generate_network_graph("06_12", save=True)
generate_network_graph("13_18", save=True)
generate_network_graph("25_29", save=True)

